In [ ]:
import json
import requests
import os
import base64
import sklearn

from eval_cus import check
import numpy as np
from sklearn.metrics.cluster import normalized_mutual_info_score as nmi
import torch


In [ ]:
url = "https://api-us.faceplusplus.com/imagepp/v1/facealbum/createalbum"
secret = "" 
key = ""

In [ ]:
k = requests.post(url, data={"api_key": key, "api_secret": secret})
print(k.text
     )

In [ ]:

face_token = json.loads(k.text)["facealbum_token"]

In [ ]:
persons = os.listdir(path)
persons

images = os.listdir(path + '/' + person)
# len(images)

In [ ]:
import pickle

pkl_file = open('faces_all.txt','rb')

faces = pickle.load(pkl_file)
# print(faces)

In [ ]:
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score
# from scipy.optimize import linear_sum_assignment as linear_assignment
from sklearn.utils.linear_assignment_ import linear_assignment
import pandas as pd

def nmi(y_true, y_pred):
    return normalized_mutual_info_score(y_true, y_pred)


def ari(y_true, y_pred):
    return adjusted_rand_score(y_true, y_pred)


def acc(y_true, y_pred):
    y_true, y_pred = pd.Series(list(y_true)), pd.Series(list(y_pred))
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    ind = linear_assignment(w.max() - w)
    return sum([w[i, j] for i, j in ind]) * 1.0 / y_pred.size

In [ ]:
faces3 = [] #list of list of faces
for j in faces:
    faces2 = []
    for k in j:
        faces2.append('custom3/' + '/'.join(k.split('/')[1:]))
    faces3.append(faces2)
print(faces3)

In [ ]:
def sim(runs, faces3):
    #add image randomly
    secret = "" 
    key = ""
    faces_all = []
    acc_all = []
    nmi_all = []
    ari_all = []
    
    for run in range(runs):
        current_faces = faces3[run]
        url = "https://api-us.faceplusplus.com/imagepp/v1/facealbum/createalbum"
        k = requests.post(url, data={"api_key": key, "api_secret": secret})
        face_token = json.loads(k.text)["facealbum_token"]
        print(f'==> Create Album count {run} \n{k.text}')
        
        url = "https://api-us.faceplusplus.com/imagepp/v1/facealbum/addimage"
        
        path = "custom_adv3"
        imageid2ground = {}
        images_stored = []
        no_faces = []
        faces = []
        
        persons = os.listdir(path)
        ground = 0

        for g, person in enumerate(persons):
            if person.startswith('.'):
                continue
            co = 0

            print(f'curr:{ground} person:{person}')

            for image in current_faces:
                lis = image.split('/')
                if lis[1] == person:
                    print(f'curr:{ground} person:{lis[1]} co:{co}')
                    with open(lis[0] + '/' + lis[1] + '/'+ lis[2], "rb") as img_file:
                        send_img64 = base64.b64encode(img_file.read())
                    response = requests.post(url, data={"api_key": key, "api_secret": secret, "facealbum_token": face_token, "image_base64":send_img64})
            #         print(response.text)
                    if not response.ok:
                        print(response.text)
                        no_faces.append(lis[0] + '/' + lis[1] + '/'+ lis[2])
                        continue
                    else:
                        faces.append(lis[0] + '/' + lis[1] + '/'+ lis[2])

                    imgid = json.loads(response.text)["image_id"]
                    imageid2ground[imgid] = ground
                    images_stored.append(str(lis[0] + '/' + lis[1] + '/'+ lis[2]))
                    co += 1
            ground += 1
            
        faces_all.append(faces)
        print(f'==> Grouping faces {run} ')
        url = "https://api-us.faceplusplus.com/imagepp/v1/facealbum/groupface"
        response = requests.post(url, data={"api_key": key, "api_secret": secret, "facealbum_token": face_token})
        print(response.text)
        task_id = json.loads(response.text)["task_id"]
        
        print(f'==> Getting clusters {run} ')
        start_token = ''
        predicted_clusters = []
        ground_truth = []
        url = "https://api-us.faceplusplus.com/imagepp/v1/facealbum/getalbumdetail"

        response = requests.post(url, data={"api_key": key, "api_secret": secret, "facealbum_token": face_token, "task_id": task_id})
        start_token = json.loads(response.text)["next_token"]
        processed = defaultdict(lambda: False)
        

        repeat = defaultdict(int)

        while(True):
            print(len(json.loads(response.text)["faces"]))

            for j in json.loads(response.text)["faces"]:
                if not processed[j["image_id"]]:
                    predicted_clusters.append(int(j["group_id"]))
                    ground_truth.append(imageid2ground[j["image_id"]])
                    processed[j["image_id"]] = True
                else:
                    repeat[j["image_id"]] += 1

            try:
                start_token = json.loads(response.text)["next_token"]
            except:
                print(response.text)
                break

            response = requests.post(url, data={"api_key": key, "api_secret": secret, "facealbum_token": face_token, "task_id": task_id, "start_token": start_token})
        acc_all.append(acc(ground_truth, predicted_clusters))
        nmi_all.append(nmi(ground_truth, predicted_clusters))
        ari_all.append(ari(ground_truth, predicted_clusters))
        
    
    pkl_file=open('faces_2.txt','wb')
    pickle.dump(faces_all, pkl_file)
    print(acc_all)
    print(nmi_all)
    print(ari_all)
    return acc_all, nmi_all, ari_all, faces_all, ground_truth, predicted_clusters
        
        

In [ ]:
acc, nmi, ari, faces_all, ground_truth, predicted_clusters = sim(10, faces3)

In [ ]:
acc, nmi, ari, ground_truth, predicted_clusters

In [ ]:
#add image
url = "https://api-us.faceplusplus.com/imagepp/v1/facealbum/addimage"
path = "custom3"
imageid2ground = {}
images_stored = []
no_faces = []
faces = []

persons = os.listdir(path)
ground = 0

for g, person in enumerate(persons):
    if person.startswith('.'):
        continue
    co = 0
    
    print(f'curr:{ground} person:{person}')
    
    for image in faces2:
        lis = image.split('/')
        if lis[1] == person:
            print(f'curr:{ground} person:{lis[1]} co:{co}')
            with open(lis[0] + '/' + lis[1] + '/'+ lis[2], "rb") as img_file:
                send_img64 = base64.b64encode(img_file.read())
            response = requests.post(url, data={"api_key": key, "api_secret": secret, "facealbum_token": face_token, "image_base64":send_img64})
    #         print(response.text)
            if not response.ok:
                print(response.text)
                no_faces.append(lis[0] + '/' + lis[1] + '/'+ lis[2])
                continue
            else:
                faces.append(lis[0] + '/' + lis[1] + '/'+ lis[2])

            imgid = json.loads(response.text)["image_id"]
            imageid2ground[imgid] = ground
            images_stored.append(str(lis[0] + '/' + lis[1] + '/'+ lis[2]))
            co += 1
    ground += 1

In [ ]:

imageid2ground

In [ ]:
len(imageid2ground)

In [ ]:
np.save('images_stored.npy', images_stored)


In [ ]:
print(len(imageid2ground))

In [ ]:
print(len(images_stored))

In [ ]:
url = "https://api-us.faceplusplus.com/imagepp/v1/facealbum/getalbumdetail"
response = requests.post(url, data={"api_key": key, "api_secret": secret, "facealbum_token": face_token})
print(response.text)

In [ ]:
url = "https://api-us.faceplusplus.com/imagepp/v1/facealbum/groupface"
response = requests.post(url, data={"api_key": key, "api_secret": secret, "facealbum_token": face_token})
print(response.text)


In [ ]:
task_id = json.loads(response.text)["task_id"]

In [ ]:
url = "https://api-us.faceplusplus.com/imagepp/v1/facealbum/getalbumdetail"
response = requests.post(url, data={"api_key": key, "api_secret": secret, "facealbum_token": face_token, "task_id": task_id})
print(response.text)

In [ ]:
from collections import defaultdict

start_token = ''
predicted_clusters = []
ground_truth = []
url = "https://api-us.faceplusplus.com/imagepp/v1/facealbum/getalbumdetail"

response = requests.post(url, data={"api_key": key, "api_secret": secret, "facealbum_token": face_token, "task_id": task_id})
start_token = json.loads(response.text)["next_token"]
processed = defaultdict(lambda: False)

repeat = defaultdict(int)

while(True):
    print(len(json.loads(response.text)["faces"]))
    
    for j in json.loads(response.text)["faces"]:
        if not processed[j["image_id"]]:
            predicted_clusters.append(int(j["group_id"]))
            ground_truth.append(imageid2ground[j["image_id"]])
            processed[j["image_id"]] = True
        else:
            repeat[j["image_id"]] += 1
        
    try:
        start_token = json.loads(response.text)["next_token"]
    except:
        print(response.text)
        break
        
    response = requests.post(url, data={"api_key": key, "api_secret": secret, "facealbum_token": face_token, "task_id": task_id, "start_token": start_token})
        


In [ ]:
# predicted_clusters = []
# ground_truth = []

# print(len(json.loads(response.text)["faces"]))
# for j in json.loads(response.text)["faces"]:
#     predicted_clusters.append(int(j["group_id"]))
#     ground_truth.append(imageid2ground[j["image_id"]])

In [ ]:
# url = "https://api-us.faceplusplus.com/imagepp/v1/facealbum/getalbumdetail"
# start_token = 0
# predicted_clusters = []
# ground_truth = []

# while True:
#     if start_token:
#         response = requests.post(url, data={"api_key": key, "api_secret": secret, "facealbum_token": face_token, "task_id": task_id, "start_token":start_token})
#         try:
#             json.loads(response.text)["next_token"]
#         except:
#             print(response.text)
#             break
#     else:
#         response = requests.post(url, data={"api_key": key, "api_secret": secret, "facealbum_token": face_token, "task_id": task_id})
    
# #     start_token = json.loads(response.text)["next_token"]
#     for j in json.loads(response.text)["faces"]:
#         predicted_clusters.append(int(j["group_id"]))
#         ground_truth.append(imageid2ground[j["image_id"]])
    
# # print(response.text)

In [ ]:
repeat

In [ ]:
print(len(predicted_clusters))
print(predicted_clusters.count(0))

In [ ]:
print(len(persons))

In [ ]:
print(len(ground_truth))
print(predicted_clusters.count(0))

In [ ]:
# clas = len(np.unique(ground_truth))
# ground_truth = torch.tensor(ground_truth).cuda()
# predicted_clusters = torch.tensor(predicted_clusters).cuda()

# p = check(ground_truth, predicted_clusters, num_classes=clas, class_names = [0]*clas)
# print(p)

In [ ]:
# print(nmi(ground_truth, predicted_clusters))

In [ ]:
print(np.unique(ground_truth))

In [ ]:
print(np.unique(predicted_clusters))

In [ ]:
# zip(ground_truth, predicted_clusters)

In [ ]:
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score
# from scipy.optimize import linear_sum_assignment as linear_assignment
from sklearn.utils.linear_assignment_ import linear_assignment
import pandas as pd

def nmi(y_true, y_pred):
    return normalized_mutual_info_score(y_true, y_pred)


def ari(y_true, y_pred):
    return adjusted_rand_score(y_true, y_pred)


def acc(y_true, y_pred):
    y_true, y_pred = pd.Series(list(y_true)), pd.Series(list(y_pred))
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    ind = linear_assignment(w.max() - w)
    return sum([w[i, j] for i, j in ind]) * 1.0 / y_pred.size

In [ ]:
acc(ground_truth, predicted_clusters)

In [ ]:
nmi(ground_truth, predicted_clusters)

In [ ]:
ari(ground_truth, predicted_clusters)